In [1]:
import numpy as np
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint

import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)


Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [2]:
import keras
feature_len = 768
max_go_len = 128
max_seq_len = 1000
max_node_len = 32
Node2Vec_dim = 300

In [3]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [4]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, dot, Flatten, CuDNNLSTM, Add
from keras.layers.merge import concatenate

def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu", padding='same')(y)

    y = Conv1D(con1d_filters, 1, activation="relu", padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(y)

    x3 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x4 = Conv1D(con1d_filters, 1, activation="relu", padding='same')(input_tensor)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(2)(mix0)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y

In [5]:
def build_cnn_gru_model(input_x, con_filters, gru_units):
    x = inception_block(input_x,con_filters )
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(gru_units, return_sequences=True))(input_x)
    x_gru = Dropout(0.3)(x_gru)
     
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x_gru)
    x = Concatenate()([ x_a, x_b, x_c, x_gru_c, x_gru_b,  x_gru_a])
    x = Dense(256)(x)
    return x


def build_cnn_model(input_x, con_filters, gru_units):
    x = inception_block(input_x,con_filters )
    x = Dropout(0.3)(x)
     
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    
    x = Concatenate()([ x_a, x_b, x_c])
    x = Dense(256)(x)
    return x 


def build_model():
    con_filters = 128
    gru_units = 64
    left_input_go = Input(shape=(max_go_len,feature_len))
    right_input_go = Input(shape=(max_go_len,feature_len))
    
    left_input_node = Input(shape=(max_node_len,Node2Vec_dim))
    right_input_node = Input(shape=(max_node_len,Node2Vec_dim))
    
    
    left_input_seq = Input(shape=(max_seq_len,20))
    right_input_seq = Input(shape=(max_seq_len,20))
    
    
    
    
    
    

    left_x_go = build_cnn_gru_model(left_input_go, con_filters, gru_units)
    right_x_go = build_cnn_gru_model(right_input_go, con_filters,gru_units)
    
    left_x_seq = build_cnn_gru_model(left_input_seq, con_filters, gru_units)
    right_x_seq = build_cnn_gru_model(right_input_seq, con_filters, gru_units)
    
    left_x_node = build_cnn_gru_model(left_input_node, con_filters, gru_units)
    right_x_node = build_cnn_gru_model(right_input_node, con_filters,gru_units)
    
      
     
    
    
   
   
    x =   Concatenate()([left_x_go  , right_x_go, left_x_node,  right_x_node, left_x_seq, right_x_seq])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu')(x)
  
    x = Dense(1)(x)
    output = Activation('sigmoid')(x)
    # model = Model([left_input_go, right_input_go], output)
  
    model = Model([left_input_go, right_input_go, left_input_node, right_input_node,     left_input_seq, right_input_seq], output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


 

model = build_model()
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 768)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 128, 768)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 32, 300)      0                                            
___________________________________________________________________________

In [6]:
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

 

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq


In [7]:
from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

In [8]:
prot2seq = load_dict('prot2seq.pkl')

In [9]:
protein2onehot = {}
for key, value in prot2seq.items():
    protein2onehot[key] =  protein_one_hot(value, max_seq_len) 

In [10]:
with open('Wnt_proteinA.txt', 'r') as f:
    all_lines_A = f.readlines()
with open('Wnt_proteinB.txt', 'r') as f:
    all_lines_B = f.readlines()

In [11]:
ppi_pairs = []
for i in range(96):
    p1 = all_lines_A[i*2].rstrip()[1:]
    p2 = all_lines_B[i*2].rstrip()[1:]
    ppi_pair = p1 + '\t' + p2 + '\t1\n'
    ppi_pairs.append(ppi_pair)

In [12]:
ppi_go_terms = load_dict('prot2go.pkl')

In [13]:
prot2emb = {}
for key, value in ppi_go_terms.items():
    X_go1 =  np.zeros((1,768))
    allgos = value.split(';') 
    allgos = list(set(allgos))
    count = 0
    for  go in  allgos:
        if go.startswith('GO'):
            feature = np.load('../../ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
            if count + feature.shape[0] > max_go_len:
                break
            X_go1 = np.concatenate((X_go1,feature ))    
            count += feature.shape[0]
    prot2emb[key] =  X_go1[1:]   


In [14]:
class TermVectors:
    def __init__(self):
        self.termVectorDict=dict()
        self.terms=None
        self.vectors=None

    def parse_term_embedding_file(self,file_path):
        with open(file_path) as f:
            lines=f.readlines()
            self.vectors=lines[1:] # 第一个是term的数量，不是termID
            terms = []
            for line in lines[1:]:
                term=line.split()[0]
                terms.append(term)
            self.terms = terms

    def str_to_vector(self,Str):
        Str = Str.strip('\n')
        nums = Str.split()
        vec = []
        for num in nums:
            vec.append(float(num))
        return np.array(vec)

    def construct_term_vector_dict(self):
        for term in self.terms:
            termindex = self.terms.index(term)
            line=self.vectors[termindex]
            s =line[len(term):].lstrip(' ')
            s_vec =self.str_to_vector(s)
            self.termVectorDict[term]=s_vec

In [15]:
# construct Node2Vec termVector
BP_TERM_EMB_FILE_PATH = '/home/xhh/PPI/Node2Vec/EDGELIST/BP.emb'
CC_TERM_EMB_FILE_PATH = '/home/xhh/PPI/Node2Vec/EDGELIST/CC.emb'
MF_TERM_EMB_FILE_PATH = '/home/xhh/PPI/Node2Vec/EDGELIST/MF.emb'
Node2Vec_dim = 300


BPTermVectors = TermVectors()
BPTermVectors.parse_term_embedding_file(BP_TERM_EMB_FILE_PATH)
BPTermVectors.construct_term_vector_dict()
print(len(BPTermVectors.termVectorDict))

CCTermVectors = TermVectors()
CCTermVectors.parse_term_embedding_file(CC_TERM_EMB_FILE_PATH)
CCTermVectors.construct_term_vector_dict()
print(len(CCTermVectors.termVectorDict))

MFTermVectors = TermVectors()
MFTermVectors.parse_term_embedding_file(MF_TERM_EMB_FILE_PATH)
MFTermVectors.construct_term_vector_dict()
print(len(MFTermVectors.termVectorDict))

29699
4202
11148


In [16]:
# read go.obo obtain ontology type
id_type_dicts = {}
obo_file = '/home/xhh/PPI/cross-species/go.obo'
fp=open(obo_file,'r')
obo_txt=fp.read()
fp.close()
obo_txt=obo_txt[obo_txt.find("[Term]")-1:]
obo_txt=obo_txt[:obo_txt.find("[Typedef]")]
# obo_dict=parse_obo_txt(obo_txt)
id_type_dicts = {}
for Term_txt in obo_txt.split("[Term]\n"):
    if not Term_txt.strip():
        continue
    name = ''
    ids = []
    for line in Term_txt.splitlines():
        if   line.startswith("id: "):
            ids.append(line[len("id: "):])     
        elif line.startswith("namespace: "):
             name=line[len("namespace: "):]
        elif line.startswith("alt_id: "):
            ids.append(line[len("alt_id: "):])
    
    for t_id in ids:
        id_type_dicts[t_id] = name

In [17]:
prot2nodevec = {}
for key, value in ppi_go_terms.items():
    X_go1 =  np.zeros((1,Node2Vec_dim))
    allgos = value.split(';') 
    allgos = list(set(allgos))
    count = 0
    for  go in  allgos:
        if go.startswith('GO'):
            if id_type_dicts[go] == 'biological_process':
                termVectors = BPTermVectors
                term_ID=go[3:].lstrip('0')
                if term_ID in termVectors.termVectorDict.keys():
                    feature = termVectors.termVectorDict[term_ID].reshape(1, Node2Vec_dim)
                else:
                    feature = np.zeros((1,Node2Vec_dim))
                
            elif id_type_dicts[go] == 'cellular_component':
                termVectors = CCTermVectors
                term_ID=go[3:].lstrip('0')
                if term_ID in termVectors.termVectorDict.keys():
                    feature = termVectors.termVectorDict[term_ID].reshape(1, Node2Vec_dim)
                else:
                    feature = np.zeros((1,Node2Vec_dim))
            elif id_type_dicts[go] == 'molecular_function':
                termVectors = MFTermVectors
                term_ID=go[3:].lstrip('0')
                if term_ID in termVectors.termVectorDict.keys():
                    feature = termVectors.termVectorDict[term_ID].reshape(1, Node2Vec_dim)
                else:
                    feature = np.zeros((1,Node2Vec_dim))
            else:
                 
                feature = np.zeros((1,Node2Vec_dim))


            if count + feature.shape[0] > max_node_len:
                break
            X_go1 = np.concatenate((X_go1,feature ))    
            count += feature.shape[0]
    prot2nodevec[key] =  X_go1[1:]  

In [18]:
X_go1 = np.empty((len(ppi_pairs), max_go_len,768))
         
X_go2 = np.empty((len(ppi_pairs), max_go_len,768))
y = np.empty((len(ppi_pairs)))
        
X_seq1 = np.empty((len(ppi_pairs), max_seq_len,20))
X_seq2 = np.empty((len(ppi_pairs), max_seq_len,20))
X_node1 = np.empty((len(ppi_pairs), max_node_len,Node2Vec_dim))        
X_node2 = np.empty((len(ppi_pairs), max_node_len,Node2Vec_dim))

for i, ppi_pair in enumerate(ppi_pairs):
     
    p1, p2, label = ppi_pair.rstrip().split('\t')
    if label == '1':
        y[i] = 1
    else:
        y[i] = 0

    feature = prot2emb[p1]
    X_go1[i,:feature.shape[0]] = feature

    feature = prot2emb[p2]
    X_go2[i,:feature.shape[0]] = feature

    X_seq1[i] =  protein2onehot[p1]
    X_seq2[i] =  protein2onehot[p2]
    
    prot1emb_node = prot2nodevec[p1]
    X_node1[i,:prot1emb_node.shape[0]] = prot1emb_node
            
    prot2emb_node = prot2nodevec[p2]
    X_node2[i,:prot2emb_node.shape[0]] = prot2emb_node

In [27]:
model.load_weights('../../SC_CV/node2vec_bert_GoplusSeq2-0.hdf5')

y_pred_prob = model.predict([X_go1, X_go2, X_node1, X_node2, X_seq1, X_seq2])
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
y_pred = (y_pred_prob > 0.5)
y_test = y
acc = accuracy_score(y_test, y_pred)
print(acc)

1.0


In [20]:
# np.save('pred_results',y_pred)

In [28]:
y_pred_prob

array([[0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],
       [0.5041273],


In [22]:
#ref: LightGBM-PPI: Predicting protein-protein interactions through LightGBM with multi-information fusion